In [1]:
import extra_keras_metrics
import keras.utils
import numpy as np
import tensorflow as tf
from keras import backend as B
from keras.layers import BatchNormalization, Dense
from keras.models import Sequential
from keras_tqdm import TQDMNotebookCallback as ktqdm
from matplotlib import pyplot as plt
from plot_keras_history import plot_history
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
#from tensorflow.python.keras.metrics import Metric
from tqdm import tqdm_notebook as tqdm

from tensorflow.python.keras.metrics import AUC
from tensorflow.python.keras.metrics import Accuracy
from tensorflow.python.keras.metrics import BinaryAccuracy
from tensorflow.python.keras.metrics import BinaryCrossentropy
from tensorflow.python.keras.metrics import CategoricalAccuracy
from tensorflow.python.keras.metrics import CategoricalCrossentropy
from tensorflow.python.keras.metrics import CategoricalHinge
from tensorflow.python.keras.metrics import CosineSimilarity
from tensorflow.python.keras.metrics import FalseNegatives
from tensorflow.python.keras.metrics import FalsePositives
from tensorflow.python.keras.metrics import Hinge
from tensorflow.python.keras.metrics import KLDivergence
from tensorflow.python.keras.metrics import LogCoshError
from tensorflow.python.keras.metrics import Mean
from tensorflow.python.keras.metrics import MeanAbsoluteError
from tensorflow.python.keras.metrics import MeanAbsolutePercentageError
from tensorflow.python.keras.metrics import MeanIoU
from tensorflow.python.keras.metrics import MeanRelativeError
from tensorflow.python.keras.metrics import MeanSquaredError
from tensorflow.python.keras.metrics import MeanSquaredLogarithmicError
from tensorflow.python.keras.metrics import MeanTensor
from tensorflow.python.keras.metrics import Metric
from tensorflow.python.keras.metrics import Poisson
from tensorflow.python.keras.metrics import Precision
from tensorflow.python.keras.metrics import PrecisionAtRecall
from tensorflow.python.keras.metrics import Recall
from tensorflow.python.keras.metrics import RecallAtPrecision
from tensorflow.python.keras.metrics import RootMeanSquaredError
from tensorflow.python.keras.metrics import SensitivityAtSpecificity
from tensorflow.python.keras.metrics import SparseCategoricalAccuracy
from tensorflow.python.keras.metrics import SparseCategoricalCrossentropy
from tensorflow.python.keras.metrics import SparseTopKCategoricalAccuracy
from tensorflow.python.keras.metrics import SpecificityAtSensitivity
from tensorflow.python.keras.metrics import SquaredHinge
from tensorflow.python.keras.metrics import Sum
from tensorflow.python.keras.metrics import TopKCategoricalAccuracy
from tensorflow.python.keras.metrics import TrueNegatives
from tensorflow.python.keras.metrics import TruePositives
from tensorflow.python.keras.metrics import binary_accuracy
from tensorflow.python.keras.metrics import categorical_accuracy
from tensorflow.python.keras.metrics import deserialize
from tensorflow.python.keras.metrics import get
from tensorflow.python.keras.metrics import serialize
from tensorflow.python.keras.metrics import sparse_categorical_accuracy
from tensorflow.python.keras.metrics import sparse_top_k_categorical_accuracy
from tensorflow.python.keras.metrics import top_k_categorical_accuracy

Using TensorFlow backend.


In [2]:
class Hephaestus:
    def __init__(self, model, x, y):
        self._model = model
        self.x = x
        self.y = y
        self.histories = []
    def kfold_x_validation(self, epochs = 100, batch = 64, shuffle = True, folds = 5):
        test_len = int(len(self.x) / folds)
        current_train_x, current_train_y = self.x, self.y
        current_test_x, current_test_y = [], []
        for i in tqdm(range(folds)):
            folded_model = self._model
            if folds == 1:
                current_test_x, current_test_y = current_train_x[:int(test_len*0.25)], current_train_y[:int(test_len*0.25)]
                current_train_x, current_train_y = current_train_x[int(test_len*0.25):], current_train_y[int(test_len*0.25):]
            else:
                current_test_x, current_test_y = current_train_x[:test_len], current_train_y[:test_len]
                current_train_x, current_train_y = current_train_x[test_len:], current_train_y[test_len:]
            history = folded_model.fit(np.array(current_train_x),
                      np.array(current_train_y),
                      epochs=epochs,
                      shuffle=shuffle,
                      batch_size=batch,
                      verbose=0,
                      callbacks=[ktqdm(metric_format="{name}: {value:e}")],
                      validation_data=(np.array(current_test_x), np.array(current_test_y))
                     )
            self.histories.append(history)
            current_train_x += current_test_x
            current_train_y += current_test_y
            current_test_x, current_test_y = [], []
        return self.histories